# Data Preprocess for the Dataset

In this session, we will do the following things:
1. Download official dataset
2. Prepare training and testing data (Road_River Altogether)
3. Delete the data that are low quality
4. Prepare 2 domain-specific datasets (Road and River )
5. Copy the data to the model directory

In [ ]:
import os
import shutil

In [ ]:
# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('../pytorch-CycleGAN-and-pix2pix'):
    os.chdir('../')
    !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
    os.chdir('./dataset')	# change directory back to `./dataset`
else:
    print('pytorch-CycleGAN-and-pix2pix is already cloned.')

In [ ]:
!pwd

# 1. Download Dataset
The script will download the dataset if you haven't downloaded it yet.

In [ ]:
!bash ../scripts/download_official_dataset.sh

# 2. Prepare Training and Testing Data

we will cover the following steps: (Road River Altogether)
- Prepare training data
- Prepare public and private testing data

## Prepare Training Data

The Training dataset contains two subfolder:
- label_img: contains the draft images
- img: contains the corresponding ground truth images

In [ ]:
import zipfile

train_dataset_zip = '34_Competition 1_Training dataset.zip'

# unzip the train_dataset_zip
with zipfile.ZipFile(train_dataset_zip, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
train_dir = 'training_dataset'

# rename the extracted folder
os.rename('Training dataset', train_dir)

In [ ]:
train_dir = './training_dataset'
print(os.listdir(train_dir))

### Rename the subfolders as trainA and trainB
mapping the folder name to the model input:
- `training_dataset/label_img` -> `training_dataset/trainA`
- `training_dataset/img` -> `training_dataset/trainB`

In [ ]:
# rename the subfolders
os.rename(train_dir + '/label_img', train_dir + '/trainA')
os.rename(train_dir + '/img', train_dir + '/trainB')

## Prepare Public and Private Testing Data

1. The extracted zip file only contains `label_img` folder
2. so we need to create the parent folder `testing_dataset`
3. and move the `label_img` folder to `testing_dataset`

In [ ]:
import zipfile

public_testing_dataset_zip = '34_Competition 1_public testing dataset.zip'
private_testing_dataset_zip = '34_Competition 1_Private Test Dataset.zip'
test_dir = 'testing_dataset'

# unzip the public testing dataset
with zipfile.ZipFile(public_testing_dataset_zip, 'r') as zip_ref:
	zip_ref.extractall(test_dir)

# unzip the private testing dataset
with zipfile.ZipFile(private_testing_dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(test_dir)

In [ ]:
test_dir = 'testing_dataset'
print(os.listdir(test_dir))

### Rename the subfolder as testA

since the ground truth images are not provided, we just need to rename the folder `label_img` as `testA`

mapping the folder name to the model input:
- `testing_dataset/label_img` -> `testing_dataset/testA`

In [ ]:
os.rename(test_dir + '/label_img', test_dir + '/testA')
print(os.listdir(test_dir))

# 3. Data Filter (Optional)

At this session, we filter the low-quality images in training data.

We have provided a text file that contains the list of low-quality images' name.

In [ ]:
file_to_delete = []
with open('file_to_delete.txt', 'r') as file:
    # Loop through reach line in the file
    for line in file:
        if '#' in line:
            continue
        # Remove any trailing whitespace (like newlines)
        line = line.strip()
        file_to_delete.append(line)
# print(file_to_delete)

In [ ]:
# Original file size
print('Original file size')
for subfolder in os.listdir(train_dir):
    print(f"{subfolder} size: {len(os.listdir(os.path.join(train_dir, subfolder)))}")


# Filter
for subfolder in os.listdir(train_dir):
    for image_name in os.listdir(os.path.join(train_dir, subfolder)):
        prefix_name = image_name.split('.')[0]
        if prefix_name in file_to_delete:
            os.remove(os.path.join(train_dir, subfolder, image_name))
            
# New file size
print('After filtering')
for subfolder in os.listdir(train_dir):
    print(f"{subfolder} size: {len(os.listdir(os.path.join(train_dir, subfolder)))}")

# 4. Prepare 2 domain-specific datasets (Road and River)

## Extract the Images from Raw Training Data
Each `trainA` and `trainB` subfolders contains 2 types of images:
- River images(e.g. TRA_RI_1000000.png)
- Road images(e.g. TRA_RO_1000000.png)

so we need to create 2 folders:
- River (contains `trainA` and `trainB` subfolders, each contains river images)
- Road (contains `trainA` and `trainB` subfolders, each contains road images)

In [ ]:
train_dir = 'training_dataset'
train_river_dir = 'train_RIVER'
train_road_dir = 'train_ROAD'

# create the folders
if not os.path.exists(train_river_dir):
	os.makedirs(train_river_dir)
if not os.path.exists(train_road_dir):
	os.makedirs(train_road_dir)

for subdir in os.listdir(train_dir):
	# create the subfolders if not exist
	if not os.path.exists(train_river_dir + '/' + subdir):
		os.makedirs(train_river_dir + '/' + subdir)
	if not os.path.exists(train_road_dir + '/' + subdir):
		os.makedirs(train_road_dir + '/' + subdir)
	
	# move or copy the files
	for file in os.listdir(train_dir + '/' + subdir):
		if '_RI_' in file:
			shutil.copy(train_dir + '/' + subdir + '/' + file, train_river_dir + '/' + subdir + '/' + file)
		elif '_RO_' in file:
			shutil.copy(train_dir + '/' + subdir + '/' + file, train_road_dir + '/' + subdir + '/' + file)
		else:
			print('ERROR: file name not recognized: ' + file)

## Extract the Images from Raw Testing Data
`testing_dataset/testA` subfolder contains 2 types of images:
- River images(e.g. PUB_RI_1000000.png or PRI_RI_1000000.png)
- Road images(e.g. PUB_RO_1000459.png or PRI_RO_1000459.png)

so we need to create 2 folders:
- test_RIVER (contains `testA` subfolders, only contains river images)
- test_ROAD (contains `testA` subfolders, only contains road images)

In [ ]:
import os
import shutil

test_dir = 'testing_dataset'
test_river_dir = 'test_RIVER'
test_road_dir = 'test_ROAD'

# create the folders
if not os.path.exists(test_river_dir):
	os.makedirs(test_river_dir)
if not os.path.exists(test_road_dir):
	os.makedirs(test_road_dir)
 

for subdir in os.listdir(test_dir):
	# create the subfolders if not exist
	if not os.path.exists(test_river_dir + '/' + subdir):
		os.makedirs(test_river_dir + '/' + subdir)
	if not os.path.exists(test_road_dir + '/' + subdir):
		os.makedirs(test_road_dir + '/' + subdir)
	
	# move or copy the files
	for file in os.listdir(test_dir + '/' + subdir):
		if '_RI_' in file:
			shutil.copy(test_dir + '/' + subdir + '/' + file, test_river_dir + '/' + subdir + '/' + file)
		elif '_RO_' in file:
			shutil.copy(test_dir + '/' + subdir + '/' + file, test_road_dir + '/' + subdir + '/' + file)
		else:
			print('ERROR: file name not recognized: ' + file)

After finishing the above steps, we will have the following folder structure:
```
dataset
├── test_ROAD
│   └── testA
├── test_RIVER
│   └── testA
├── train_ROAD
│   ├── trainA
│   └── trainB
└── train_RIVER
    ├── trainA
    └── trainB
```

# 5. Copy the data to the model directory

## Copy the training data to the model input folder

move the `train_RIVER` and `train_ROAD` folders to `../pytorch-CycleGAN-and-pix2pix/datasets`

In [ ]:
# copy the folder to target folder
target_dir = '../pytorch-CycleGAN-and-pix2pix/datasets'
shutil.copytree(train_river_dir, target_dir + '/' + train_river_dir)
shutil.copytree(train_road_dir, target_dir + '/' + train_road_dir)

## Copy the testing data to the model input folder

copy the `test_RIVER` and `test_ROAD` folders to `../pytorch-CycleGAN-and-pix2pix/datasets`

In [ ]:
# copy the folder to target folder
target_dir = '../pytorch-CycleGAN-and-pix2pix/datasets'
shutil.copytree(test_river_dir, target_dir + '/' + test_river_dir)
shutil.copytree(test_road_dir, target_dir + '/' + test_road_dir)

## Align the pair images for train_ROAD and train_RIVER

In [ ]:
os.chdir('../pytorch-CycleGAN-and-pix2pix/datasets')

In [ ]:
!pwd

In [ ]:
print('Creating aligned dataset for train_ROAD...')
!python make_dataset_aligned.py --dataset-path train_ROAD

print('Creating aligned dataset for train_RIVER...')
!python make_dataset_aligned.py --dataset-path train_RIVER

After finishing the above steps, we will have the following folder structure in `pytorch-CycleGAN-and-pix2pix/datasets`:
```
datasets
├── test_ROAD
│   └── testA
├── test_RIVER
│   └── testA
├── train_ROAD
│   ├── train
│   ├── trainA
│   └── trainB
└── train_RIVER
    ├── train
    ├── trainA
    └── trainB
```